In [9]:
import os
import json

import imageio

SIZE = 256
files_and_crops = {}


def crop_and_save(img, filename, coords):
    """
    coords:[(y1,x1), (y2,x2)]
    """
    crop_img = img[coords[0][0]:coords[1][0], coords[0][1]:coords[1][1]]
    imageio.imwrite(filename, crop_img)


def crop(filename, save_directory, set_name, counter):
    image_descriptor = {}
    print('CROPPING: {}'.format(filename))
    img = imageio.imread(filename)
    shape = img.shape
    counter += 1
    fname = os.path.split(filename)[1]
    for y in range(0, shape[0]-SIZE, SIZE):
        for x in range(0, shape[1]-SIZE, SIZE):
            filename = '{}.png'.format(counter)
            save_path = os.path.join(save_directory, set_name, filename)
            crop_and_save(img, save_path, [(y, x), (y + SIZE, x + SIZE)])
            image_descriptor.update({filename: {'x1': x, 'y1': y, 'x2': x + SIZE, 'y2': y + SIZE}})
            counter += 1
            if shape[1] - SIZE < x + SIZE < shape[1]:
                save_path = os.path.join(save_directory, set_name, '{}.png'.format(counter))
                crop_and_save(img, save_path, [(y, shape[1]-SIZE), (y+SIZE, shape[1])])
                image_descriptor.update({filename: {'x1': shape[1]-SIZE, 'y1': y, 'x2': shape[1], 'y2': y + SIZE}})
                counter += 1
    if shape[0] - SIZE < y + SIZE < shape[0]:
        for x in range(0, shape[1]-SIZE, SIZE):
            save_path = os.path.join(save_directory, set_name, '{}.png'.format(counter))
            crop_and_save(img, save_path, [(shape[0]-SIZE, x), (shape[0], x+SIZE)])
            image_descriptor.update({filename: {'x1': x, 'y1': shape[0]-SIZE, 'x2': x+SIZE, 'y2': shape[0]}})
            counter += 1
    
    with open(os.path.join(save_directory, set_name, 'descriptor.json'), 'w') as fp:
        fp.write(json.dumps(image_descriptor))
    return counter


def launch(source_directory='data/TRAINING', save_directory='data/CROPPED'):
    file_list = list(filter(lambda x: x[0]!='.', sorted(os.listdir(source_directory))))[6:]  # ! REMOVE
    try:
        os.mkdir(os.path.join(save_directory, 'PNG'))
        os.mkdir(os.path.join(save_directory, 'MASK'))
    except FileExistsError:
        pass
    for (i, set_name) in enumerate(file_list):
        p_counter = 0
        m_counter = 0
        
        set_source_directory = os.path.join(source_directory, set_name)
        all_files = sorted(os.listdir(os.path.join(source_directory, set_name)))
        mask_files = list(filter(lambda x: x.endswith('_mask.png') and len(x) > 11, all_files))[:4]  # !LUL
        png_files = [x.replace('_mask', '') for x in mask_files]
        print('ORDER: ', png_files)
        for (png_file, mask_file) in zip(png_files, mask_files):
            source_png_file = os.path.join(source_directory, set_name, png_file)
            source_mask_file = os.path.join(source_directory, set_name, mask_file)
            try:
                os.mkdir(os.path.join(save_directory, 'PNG/{}'.format(set_name)))
                os.mkdir(os.path.join(save_directory, 'MASK/{}'.format(set_name)))
            except FileExistsError:
                pass
            p_counter = crop(source_png_file, save_directory, 'PNG/{}'.format(set_name), p_counter)
            m_counter = crop(source_mask_file, save_directory, 'MASK/{}'.format(set_name), m_counter)
        print('DONE: {}/100%'.format((i+1)/len(file_list) * 100))

In [12]:
def clean_black_screens(target_dir='DATA/CROPPED_AND_AUGMENTED/MASK'):
    for directory in list(filter(lambda x: not x.startswith('.'), os.listdir(target_dir))):
        for file in list(filter(lambda x: x.endswith('.png'), os.listdir(os.path.join(target_dir, directory)))):
            full_file_path = os.path.join(target_dir, directory, file)
            statinfo = os.stat(full_file_path)
            if statinfo.st_size==141:
                full_png_path = full_file_path.replace('/MASK/', '/PNG/')
                print('REMOVING: {} AND MASK'.format(full_png_path))
                os.remove(full_png_path)
                os.remove(full_file_path)

In [13]:
if __name__ == '__main__':
    #launch(source_directory='DATA/TRAIN', save_directory='DATA/CROPPED_AND_AUGMENTED')
    clean_black_screens()

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3658.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/2254.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3222.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/4774.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/2971.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/845.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/4215.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/4054.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3377.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/2376.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/1166.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/862.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/2386.png AND MASK
REMOVING: DATA

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/1965.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/4252.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/2377.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/1844.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/4587.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3218.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3438.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/1605.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/833.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3499.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/2006.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/2314.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/4418.png AND MASK
REMOVING: DAT

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3812.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/4602.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/904.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3519.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/1943.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/579.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/1332.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/1722.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/2239.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3716.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/3484.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/4052.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/E1QZENNLNLCXBKPN/2113.png AND MASK
REMOVING: DATA

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/4749.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2310.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2838.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/4803.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2218.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/4554.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/3359.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/20.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/4424.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/3890.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2173.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2795.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/4754.png AND MASK
REMOVING: DATA

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2096.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/3373.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2327.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/493.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/3454.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/4383.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2371.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/3195.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/4772.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/4575.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/4295.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2277.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/8B7JP60TYSMLOTJO/2319.png AND MASK
REMOVING: DAT

...

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/4447.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/1728.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/970.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/839.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/4497.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/3376.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/926.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/1199.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/1684.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/1091.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/4618.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/553.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/4748.png AND MASK
REMOVING: DATA/C

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/4777.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/180.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/917.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/3619.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/204.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/3166.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/3697.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/3417.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/47.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/1243.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/4612.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/4422.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/YTS6DUZYSW5KADN1/2205.png AND MASK
REMOVING: DATA/CR

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/1130.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/1253.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/3561.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/2119.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/3327.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/2075.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/943.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/1132.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/4670.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/4626.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/4698.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/823.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/AZ74029PBGFI0291/2856.png AND MASK
REMOVING: DATA

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/3515.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/4588.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/2917.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/1879.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/4765.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/1173.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/4798.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/4753.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/2191.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/3490.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/4339.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/1169.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/4622.png AND MASK
REMOVING: DA

REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/2916.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/1851.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/3564.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/3482.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/2040.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/3346.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/3453.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/2238.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/4487.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/1594.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/3287.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/3786.png AND MASK
REMOVING: DATA/CROPPED_AND_AUGMENTED/PNG/LJJ5AQ6GYW6L2M8D/4117.png AND MASK
REMOVING: DA